--- Day 5: If You Give A Seed A Fertilizer ---
You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island is the water source!" You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we ran out of sand to filter it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our food production problem. The latest Island Island Almanac just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:

seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4

The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:

50 98 2
52 50 48

The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51

With this map, you can look up the soil number required for each initial seed number:

Seed number 79 corresponds to soil number 81.
Seed number 14 corresponds to soil number 14.
Seed number 55 corresponds to soil number 57.
Seed number 13 corresponds to soil number 13.

The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.

So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

In [1]:
test_input_str = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

In [2]:
from dataclasses import dataclass
from __future__ import annotations
from typing import Optional
from tqdm import tqdm

import portion as P



@dataclass
class IntRangeMapper:
    source_range: P.closedopen
    destination_range: P.closedopen

    @property
    def value_shift(self):
        return self.destination_range.lower - self.source_range.lower

    @classmethod
    def from_str(cls, input_str: str) -> IntRangeMapper:
        destination_start, source_start, range_length = list(map(int, input_str.strip().split()))
        return cls(
            source_range=P.closedopen(source_start, source_start + range_length),
            destination_range=P.closedopen(destination_start, destination_start + range_length),
            )

    def is_in_source_range(self, source_value) -> bool:
        return source_value in self.source_range

    def is_in_destination_range(self, source_value) -> bool:
        return source_value in self.destination_range

    def get(self, source_value: int) -> Optional[int]:
        if self.is_in_source_range(source_value):
            return source_value + self.value_shift
        else:
            return None


@dataclass
class GroupIntRangeMaps:
    range_map_list: list[IntRangeMapper]
    
    @classmethod
    def from_str(cls, input_str: str) -> GroupIntRangeMaps:
        map_lines_list = input_str.strip().split("\n")
        return cls(range_map_list=sorted([IntRangeMapper.from_str(ml) for ml in map_lines_list], key=lambda x: x.source_range.lower))
    
    def get(self, source_value: int, default_value: int) -> int:
        for rng_map in self.range_map_list:
            if mapped_val:=rng_map.get(source_value):
                return mapped_val
        # if not mapped from any mapper, return default value
        return default_value


    def merge_child_maps(self, child_map_grp: GroupIntRangeMaps) -> GroupIntRangeMaps:
        parent_map_grp = self
        output_mapper_list = []
        parent_mapper_list = sorted(parent_map_grp.range_map_list, key=lambda x: x.destination_range.lower)
        child_mapper_list = sorted(child_map_grp.range_map_list, key=lambda x: x.source_range.lower)

        # find intersection of maps
        for parent_map in parent_mapper_list:
            # we will start with entire source not yet mapped and subtract from this
            parent_source_not_in_child = parent_map.source_range
            parent_destionation_not_in_child = parent_map.destination_range
            for child_map in child_mapper_list:
                # find if intersection of parent destination and child source
                intersection_range = parent_map.destination_range.intersection(child_map.source_range)
                if intersection_range:
                    parent_intersecting_source = intersection_range.apply(lambda x: (x.left, x.lower - parent_map.value_shift, x.upper - parent_map.value_shift, x.right))
                    child_intersecting_destination = intersection_range.apply(lambda x: (x.left, x.lower + child_map.value_shift, x.upper + child_map.value_shift, x.right))
                    output_mapper_list.append(IntRangeMapper(
                        source_range=parent_intersecting_source,
                        destination_range=child_intersecting_destination,
                        ))
                    
                    parent_source_not_in_child = parent_source_not_in_child - parent_intersecting_source
                    parent_destionation_not_in_child  = parent_destionation_not_in_child - intersection_range

            
            # add original parent mappings not in child
            if parent_source_not_in_child:
                output_mapper_list.append(IntRangeMapper(
                    source_range=parent_source_not_in_child,
                    destination_range=parent_destionation_not_in_child,
                ))


        # iterate once more to find maps exclusive to child
        for child_map in child_mapper_list:
            child_source_not_in_parent = child_map.source_range
            child_destination_not_in_parent = child_map.destination_range
            for parent_map in parent_mapper_list:
                # find if intersection of parent destination and child source
                intersection_range = parent_map.destination_range.intersection(child_map.source_range)

                if intersection_range:
                    child_intersecting_destination = intersection_range.apply(lambda x: (x.left, x.lower + child_map.value_shift, x.upper + child_map.value_shift, x.right))
                    
                    child_source_not_in_parent = child_source_not_in_parent - intersection_range
                    child_destination_not_in_parent  = child_destination_not_in_parent - child_intersecting_destination

            # add original child mappings not in parent
            if child_source_not_in_parent:
                output_mapper_list.append(IntRangeMapper(
                    source_range=child_source_not_in_parent,
                    destination_range=child_destination_not_in_parent,
                ))


        return GroupIntRangeMaps(range_map_list=output_mapper_list)




@dataclass
class SeedMapper:
    seed_list: list[int]
    seed_to_soil_map: GroupIntRangeMaps
    soil_to_fertilizer_map: GroupIntRangeMaps
    fertilizer_to_water_map: GroupIntRangeMaps
    water_to_light_map: GroupIntRangeMaps
    light_to_temperature_map: GroupIntRangeMaps
    temperature_to_humidity_map: GroupIntRangeMaps
    humidity_to_location_map: GroupIntRangeMaps


    @classmethod
    def parse_list_str(_cls, list_str: str) -> list[int]:
        _, list_values_str = list_str.strip().split(":")
        int_values_list = list(map(int, list_values_str.strip().split()))
        return int_values_list

    @classmethod
    def parse_map_line_str(_cls, map_line_str: str) -> dict[int, int]:
        out_dict = {}
        dest_range_start, src_range_start, range_len = list(map(int, map_line_str.strip().split()))
        for i in range(range_len):
            out_dict[src_range_start + i] = dest_range_start + i
        return out_dict

    @classmethod
    def parse_map_str(cls, map_str: str) -> dict[int, int]:
        out_dict = {}
        _, map_lines_str = map_str.strip().split(":")
        return GroupIntRangeMaps.from_str(map_lines_str)

    @classmethod
    def from_str(cls, input_str: str) -> SeedMapper:
        (
            seed_list_str,
            seed_soil_str,
            soil_fertilizer_str,
            fertilizer_water_str,
            water_light_str,
            light_temperature_str,
            temperature_humidity_str,
            humidity_location_str,
             ) = input_str.strip().split("\n\n")
        return cls(
            seed_list=cls.parse_list_str(seed_list_str),
            seed_to_soil_map=cls.parse_map_str(seed_soil_str),
            soil_to_fertilizer_map=cls.parse_map_str(soil_fertilizer_str),
            fertilizer_to_water_map=cls.parse_map_str(fertilizer_water_str),
            water_to_light_map=cls.parse_map_str(water_light_str),
            light_to_temperature_map=cls.parse_map_str(light_temperature_str),
            temperature_to_humidity_map=cls.parse_map_str(temperature_humidity_str),
            humidity_to_location_map=cls.parse_map_str(humidity_location_str),
        )

    @property
    def seed_to_location_map(self) -> GroupIntRangeMaps:
        out_map = self.seed_to_soil_map.merge_child_maps(
            self.soil_to_fertilizer_map.merge_child_maps(
                self.fertilizer_to_water_map.merge_child_maps(
                    self.water_to_light_map.merge_child_maps(
                        self.light_to_temperature_map.merge_child_maps(
                            self.temperature_to_humidity_map.merge_child_maps(
                                self.humidity_to_location_map
                            )
                        )
                    )
                )
            )
        )
        return out_map


    def get_location_by_seed(self, seed_int: int) -> int:
        return self.seed_to_location_map.get(seed_int, seed_int)


    def get_lowest_location_of_seeds(self) -> int:
        seed_locations = [self.get_location_by_seed(s) for s in self.seed_list]
        return min(seed_locations)
    
    @property
    def seed_ranges(self):
        out_range = P.empty()
        n_seed_ranges = int(len(self.seed_list) / 2)
        for pair_n in range(n_seed_ranges):
            out_range = out_range.union(P.closedopen(self.seed_list[ 2 * pair_n], self.seed_list[ 2 * pair_n] + self.seed_list[ 2 * pair_n + 1]))
        return out_range

    def get_lowest_location_from_range_of_seeds(self) -> int:
        # search more optimally by sorting by destination range and checking if any input is in range
        for seed_to_loc_map in tqdm(sorted(self.seed_to_location_map.range_map_list, key=lambda x: x.destination_range.lower)):
            seed_intersection = self.seed_ranges.intersection(seed_to_loc_map.source_range)
            if seed_intersection:
                # return the lowest value from intersection, transformed through map
                min_value = seed_intersection.lower + seed_to_loc_map.value_shift
                return min_value
        return Exception("Minimum not found")



In [3]:
test_mapper = SeedMapper.from_str(test_input_str)

In [4]:
# check values from example
assert test_mapper.get_location_by_seed(79) == 82
assert test_mapper.get_location_by_seed(14) == 43
assert test_mapper.get_location_by_seed(55) == 86
assert test_mapper.get_location_by_seed(13) == 35

assert test_mapper.get_lowest_location_of_seeds() == 35

In [5]:
with open("../inputs/day5_input.txt") as f:
    input_str = f.read()

In [6]:
seed_mapper = SeedMapper.from_str(input_str)


In [7]:
seed_mapper.get_lowest_location_of_seeds()

389056265

# Part Two

--- Part Two ---
Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:

seeds: 79 14 55 13
This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?

In [8]:
assert test_mapper.get_lowest_location_from_range_of_seeds() == 46

 40%|████      | 8/20 [00:00<00:00, 36118.87it/s]


In [9]:
seed_mapper.get_lowest_location_from_range_of_seeds()

  3%|▎         | 4/136 [00:00<00:00, 5913.72it/s]


137516820